# Manual ingest workflow, bypassing GUI

## Login

Either log in via a local config file (see [01_pipeline](./01_pipeline.ipynb)), or enter login information manually. If you are don't have your login information, contact the administrator.


In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
from adamacs.pipeline import subject, session, equipment, surgery, event, trial, imaging, behavior, model
from adamacs.ingest import session as isess
from adamacs.helpers import stack_helpers as sh
from adamacs.ingest import behavior as ibe
import pathlib
from natsort import natsorted, ns
import datajoint as dj
from rspace_client.eln import eln
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
dj.__version__

[2023-11-09 18:15:27,344][INFO]: Connecting tobiasr@172.26.128.53:3306
[2023-11-09 18:15:27,391][INFO]: Connected tobiasr@172.26.128.53:3306


'0.14.1'

### RSpace connection

In [2]:
URL=dj.config['custom'].get('rspace_URL')
API_KEY=dj.config['custom'].get('rspace_API_key')
api = eln.ELNClient(URL, API_KEY)
api.get_status()

{'message': 'OK', 'rspaceVersion': '1.89.2'}

## Activation
Next, import from `adamacs.pipeline` to activate the relevant schema.

In [3]:
from adamacs.utility import *
# from adamacs.nbgui import *
from adamacs.pipeline import subject, session, surgery, scan, equipment, model

Assign easy names for relevant tables

In [4]:
sub, lab, protocol, line, mutation, user, project, subject_genotype, subject_death = (
    subject.Subject(), subject.Lab(), subject.Protocol(), subject.Line(), 
    subject.Mutation(), subject.User(), subject.Project(), subject.SubjectGenotype(), 
    subject.SubjectDeath()
    )

some helpers

In [5]:

def get_session_dir_key_from_dir(directory):
    return [path.split('/')[-1] for path in directory]
     
def get_scan_dir_key_from_dir(directory):
    return [path.split('/')[-1] for path in directory]

def get_session_key_from_dir(string):
    result = [re.search(r'sess\S+', item).group(0) for item in string]
    return result

def get_user_initials_from_dir(string):
    result = [name[:2] for name in string]
    return result

def get_subject_key_from_dir(string):
    result = [item.split("_")[1] for item in string]
    return result

def get_date_key_from_dir(directory):
    return directory.split("_")[-1]

def get_scan_key_from_dir(string):
    result = [re.search(r'scan\S+_', item).group(0)[:-1] for item in string]
    return result

## Parse directories

In [6]:
# get content of user directory
import fnmatch
dataroot = dj.config['custom']['exp_root_data_dir'][0]
# dirs_root = [d for d in os.listdir(dataroot) if os.path.isdir(os.path.join(dataroot, d)) and '_' in d]
dirs_root = [d for d in os.listdir(dataroot) if os.path.isdir(os.path.join(dataroot, d)) and fnmatch.fnmatch(d, 'JJ*') and fnmatch.fnmatch(d, '*10-*')]
sorted_dirs_root = natsorted(dirs_root, key=get_date_key_from_dir, reverse = True)
sorted_dirs_root

['JJ_ROS-1604_2023-10-27_scan9FKSN4XD_sess9FKSN4XD',
 'JJ_ROS-1604_2023-10-27_scan9FKSMYYS_sess9FKSMYYS',
 'JJ_ROS-1604_2023-10-27_scan9FKSMSMZ_sess9FKSMSMZ',
 'JJ_ROS-1604_2023-10-19_scan9FKNVVH5_sess9FKNVVH5',
 'JJ_ROS-1604_2023-10-19_scan9FKNVPI4_sess9FKNVPI4',
 'JJ_ROS-1604_2023-10-19_scan9FKNVIUK_sess9FKNVIUK',
 'JJ_ROS-1604_2023-10-18_scan9FKNBRV9_sess9FKNBRV9',
 'JJ_ROS-1604_2023-10-18_scan9FKNBQCS_sess9FKNBQCS',
 'JJ_ROS-1627_2023-10-05_scan9FKFMG1P_sess9FKFMG1P',
 'JJ_ROS-1627_2023-10-05_scan9FKFM3IW_sess9FKFM3IW']

In [7]:
# sessi = "sess9FKW82R3"
# scansi = "scan9FKW82R3"
sessi = "sess9FI8ALGO"
scansi = "scan9FI8ALGO"

# sessi = "sess9FB2LN5C"
# scansi = "scan9FB2LN5C"

# scan9FJ842C3
# scan9FB2LN5C
# scan_key

## Session ingest

In [8]:
Project = project.fetch('project')
Equipment = equipment.Equipment().fetch('scanner')
Recording_Location = surgery.AnatomicalLocation().fetch('anatomical_location')
s2pparm = imaging.ProcessingParamSet.fetch("paramset_idx", "paramset_desc")
DLCModels = model.Model.fetch("model_name")

print(project)
print(Equipment)
print(Recording_Location)
print(s2pparm)
print(DLCModels)

*project       project_descri
+------------+ +------------+
ATN            ATN-functional
dummy          dummy         
hpc-repstab    hpc-representa
rsc-functop    rsc-functional
rsc-hpc        rsc-hippocampa
rsc-latent     rsc-contextual
sc-lgn-actvis  sc-lgn-active-
V1-oddball     v1-oddball-pre
vc-lgn-repstab vc-lgn-represe
 (Total: 9)

['bench2p' 'dummy' 'macroscope' 'mini2p_01' 'mini2p_02' 'mini2p_03'
 'mini2p_04' 'mini2p_05']
['ATN' 'Ctx' 'dCA1' 'DG' 'dummy' 'LGNV1' 'RSCa' 'RSCg' 'V1']
[array([0, 1, 2, 3, 4, 5, 6]), array(['TR: Mini2p (new, non-rigid, individual scans, reg_tiff for movie, custom classifier)',
       'TR: Mini2p (rigid, mini2p classifier, no concat, REGTIFF)',
       'TR: Trondheim Mini2p (non-rigid, built-in classifier, scan_concat)',
       'TR: Bench2p (non-rigid, custom classifier, reg_tiff for movie, scans individually)',
       'TR: Bench2p (rigid, custom bench2p classifier, individual, SAVETIF)',
       'TR: Mini2p (rigid, built-in classifier, individual)'

### Ingest Session and Scan

In [9]:
isess.ingest_session_scan(sessi, verbose=True, project_key="rsc-functop", equipment_key="mini2p_01", location_key="RSCa", software_key='ScanImage')
session.SessionSameSite.update1({'session_id': sessi, 'same_site_id': sessi})


Number of scans found: 1
[['scan9FI8ALGO']]
['RN_OPI-1681_2023-05-25_scan9FI8ALGO_sess9FI8ALGO']


/tmp/ipykernel_3694596/3791184197.py:1: UserWarning: 
Skipped existing session row: ('sess9FI8ALGO', <module 'adamacs.schemas.subject' from '/home/tobiasr/adamacs/adamacs/schemas/subject.py'>, '2023-05-25')
  isess.ingest_session_scan(sessi, verbose=True, project_key="rsc-functop", equipment_key="mini2p_01", location_key="RSCa", software_key='ScanImage')
/tmp/ipykernel_3694596/3791184197.py:1: UserWarning: 
Skipped existing ProjectSession row: ('rsc-functop', 'sess9FI8ALGO')
  isess.ingest_session_scan(sessi, verbose=True, project_key="rsc-functop", equipment_key="mini2p_01", location_key="RSCa", software_key='ScanImage')
/tmp/ipykernel_3694596/3791184197.py:1: UserWarning: 
Skipped existing SessionDirectory: ('sess9FI8ALGO', PosixPath('/datajoint-data/data/tobiasr/RN_OPI-1681_2023-05-25_scan9FI8ALGO_sess9FI8ALGO'))
  isess.ingest_session_scan(sessi, verbose=True, project_key="rsc-functop", equipment_key="mini2p_01", location_key="RSCa", software_key='ScanImage')
/tmp/ipykernel_3694596

In [10]:

scan.Scan * scan.ScanInfo * session.SessionSameSite * session.Session() & f'session_id = "{sessi}"'

session_id,scan_id,scanner,acq_software,scan_notes free-notes,nfields number of fields,nchannels number of channels,ndepths Number of scanning depths (planes),nframes number of recorded frames,nrois number of ROIs (see scanimage's multi ROI imaging),x (um) ScanImage's 0 point in the motor coordinate system,y (um) ScanImage's 0 point in the motor coordinate system,z (um) ScanImage's 0 point in the motor coordinate system,fps (Hz) frames per second - Volumetric Scan Rate,bidirectional true = bidirectional scanning,usecs_per_line microseconds per scan line,fill_fraction raster scan temporal fill fraction (see scanimage),scan_datetime datetime of the scan,scan_duration (seconds) duration of the scan,userfunction_info string argument of file consolidation userfunction 1,same_site_id,subject PyRat import uses this for earmark value,session_datetime
sess9FI8ALGO,scan9FI8ALGO,mini2p_01,ScanImage,,1,1,1,4550,0,0.0,0.0,0.0,7.57571,1,250.001,0.9,None,600.604,openfield,sess9FI8ALGO,OPI-1681,2023-05-25 00:00:00


In [11]:
query = scan.ScanPath() & 'scan_id = "' + scansi + '"'
dir_proc = query.fetch('path')[0]
print(dir_proc)

/datajoint-data/data/tobiasr/RN_OPI-1681_2023-05-25_scan9FI8ALGO_sess9FI8ALGO


In [12]:
# POPULATE!
populate_settings = {'display_progress': True, 'suppress_errors': False, 'processes': 1}
scan.ScanInfo.populate(**populate_settings) 

In [13]:
# push scan to ProcessingTask
# TODO: handle multiscan concatenation from here?
selected_s2pparms_index = 0
imaging.ProcessingTask.insert1((sessi, scansi, selected_s2pparms_index, dir_proc, 'trigger'), skip_duplicates=True)

In [14]:
imaging.ProcessingTask()

session_id,scan_id,paramset_idx,processing_output_dir output directory of the processed scan relative to root data directory,"task_mode 'load': load computed analysis results, 'trigger': trigger computation"
sess9FI8ALGO,scan9FI8ALGO,0,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-05-25_scan9FI8ALGO_sess9FI8ALGO,trigger
sess9FKFM3IW,scan9FKFM3IW,3,/datajoint-data/data/tobiasr/JJ_ROS-1627_2023-10-05_scan9FKFM3IW_sess9FKFM3IW,trigger
sess9FKW82R3,scan9FKW82R3,0,/datajoint-data/data/tobiasr/JJ_ROS-1604_2023-11-02_scan9FKW82R3_sess9FKW82R3,trigger
sess9FKW82R3,scan9FKW82R3,3,/datajoint-data/data/tobiasr/JJ_ROS-1604_2023-11-02_scan9FKW82R3_sess9FKW82R3,trigger
sess9FKYPPYS,scan9FKYPPYS,3,/datajoint-data/data/tobiasr/NK_ROS-1664_2023-11-06_scan9FKYPPYS_sess9FKYPPYS,trigger
sess9FKYPVYG,scan9FKYPVYG,3,/datajoint-data/data/tobiasr/NK_ROS-1664_2023-11-06_scan9FKYPVYG_sess9FKYPVYG,trigger


## Ingest AUX

In [16]:
aux_setup_typestr = (scan.ScanInfo() & 'scan_id = "' + scansi + '"').fetch("userfunction_info")[0]    
print(aux_setup_typestr)

openfield


In [17]:
ibe.ingest_aux(sessi,scansi,verbose=True, aux_setup_type=aux_setup_typestr)

Number of aux-files found: 1
[PosixPath('/datajoint-data/data/tobiasr/RN_OPI-1681_2023-05-25_scan9FI8ALGO_sess9FI8ALGO')]


/home/tobiasr/miniconda3/envs/datajoint-DLCbackup/lib/python3.8/site-packages/pywavesurfer/ws.py:46: RuntimeWarning: You are reading a WaveSurfer file version this module was not tested with: file version 1.0.6, latest version tested: 0.982
  warnings.warn('You are reading a WaveSurfer file version this module was not tested with: '


In [18]:
(event.Event & 'scan_id = "' + scansi + '"' & 'event_type LIKE "%mini2p%"')

session_id,scan_id,event_type,event_start_time (second) relative to recording start,event_end_time (second) relative to recording start
sess9FI8ALGO,scan9FI8ALGO,mini2p_frames,2.8852,3.01315
sess9FI8ALGO,scan9FI8ALGO,mini2p_frames,3.0172,3.14515
sess9FI8ALGO,scan9FI8ALGO,mini2p_frames,3.1492,3.27715
sess9FI8ALGO,scan9FI8ALGO,mini2p_frames,3.2812,3.40915
sess9FI8ALGO,scan9FI8ALGO,mini2p_frames,3.4132,3.54115
sess9FI8ALGO,scan9FI8ALGO,mini2p_frames,3.5452,3.67315
sess9FI8ALGO,scan9FI8ALGO,mini2p_frames,3.6772,3.80515
sess9FI8ALGO,scan9FI8ALGO,mini2p_frames,3.8092,3.93715
sess9FI8ALGO,scan9FI8ALGO,mini2p_frames,3.9412,4.06915
sess9FI8ALGO,scan9FI8ALGO,mini2p_frames,4.0732,4.20115


In [ ]:
# time_differences = np.diff((event.Event & 'scan_id = "' + scansi + '"' & 'event_type LIKE "%reward%"').fetch("event_start_time"))

# plt.figure(figsize=(10, 6))
# plt.hist(time_differences, bins=range(min(time_differences.astype(int)), max(time_differences.astype(int)) + 10, 1), edgecolor='black')
# plt.title('Histogram of Time Differences Between Consecutive Events')
# plt.xlabel('Time Difference (seconds)')
# plt.ylabel('Frequency')
# plt.show()

## Ingest BPOD


In [ ]:
ibe.ingest_bpod(sessi,scansi,verbose=False, aux_setup_type=aux_setup_typestr)

### get BPOD object to play around

In [ ]:
from adamacs.paths import get_imaging_root_data_dir, get_experiment_root_data_dir
from adamacs.ingest import bpod
from element_interface.utils import find_full_path

scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0]
bpod_path_relative = (scan.ScanPath & scan_key).fetch("path")[0]

bpod_path_full = list(find_full_path(
        get_experiment_root_data_dir(), bpod_path_relative
        ).glob("*mat"))[0]

In [ ]:
bpod_object = bpod.Bpodfile(bpod_path_full)

In [ ]:
bpod_object.session_data["Info"]

In [ ]:
bpod_object.session_data["MousePos"]['PreStimDLC_live'][10]['right_ear']

In [ ]:
bpod_object.trial_data[0]['Events']


In [ ]:

trials = bpod_object.trial_data
BNC1Low_events = [(i, trial['Events'].get('BNC1Low')) for i, trial in enumerate(trials) if 'BNC1Low' in trial['Events']]
print(BNC1Low_events)
# BNC1Low_events[0][0]

In [ ]:

trials[0]['Events']

In [ ]:
bpod_object.session_data["RawEvents"]["Trial"][1]["Events"]

In [ ]:
bpod_object.session_data["RawEvents"]["Trial"][0]["Events"]["BNC1High"]

In [ ]:
bpod_object.session_data["RawEvents"]["Trial"][-2]["Events"]

In [ ]:
BNC1Low_events = [(i, event.get('BNC1Low')) for i, event in enumerate(events) if 'BNC1Low' in event]

In [ ]:
BNC1Low_events

In [ ]:
bpod_object.session_data["StimPos"].keys()

In [ ]:
bpod_object.trial_data[34]['Events']

In [ ]:
trials = bpod_object.trial_data
SoftCode15_events = [(i, trial['Events'].get('SoftCode15')) for i, trial in enumerate(trials) if 'SoftCode15' in trial['Events']] # get all trials that have a BNC1Low event. Returns a list of tuples (trial number, event time)

SoftCode15_events_times = [x[1] for x in SoftCode15_events]

plt.hist(SoftCode15_events_times, bins=1000, edgecolor='black')
plt.xlim([0, 1])
plt.title('Histogram of softcode 15 event-to-trial-beginning time')
plt.xlabel('[s]')
plt.ylabel('Frequency')
# plt.xlim([0, 1])
plt.show()

In [ ]:
trials = bpod_object.trial_data
SoftCode10_events = [(i, trial['Events'].get('SoftCode10')) for i, trial in enumerate(trials) if 'SoftCode10' in trial['Events']] # get all trials that have a BNC1Low event. Returns a list of tuples (trial number, event time)

SoftCode10_events_times = [x[1] for x in SoftCode10_events]

plt.hist(SoftCode10_events_times, edgecolor='black')
plt.xlim([0, 1])
plt.title('Histogram of softcode 10 event-to-trial-beginning time')
plt.xlabel('[s]')
plt.ylabel('Frequency')
# plt.xlim([0, 1])
plt.show()

In [ ]:
SoftCode10_events_times

In [ ]:
1/60

In [ ]:
# get the times of the tone onsets
aux_tone = (event.Event & 'scan_id = "' + scansi + '"' & 'event_type LIKE "%vis%"').fetch("event_start_time")
# create a time series of ones, using the same length as the tone onsets
onesfive = np.ones_like(aux_tone) -.5
# plot the time series of ones vs the tone onsets
axs[0].scatter(aux_tone, onesfive)

# get the times of the trial start events
bpod_trial = bpod_object.session_data["TrialStartTimestamp"]
# create a time series of ones, using the same length as the trial start events
ones = np.ones_like(bpod_trial)
# plot the time series of ones vs the trial start events
axs[1].scatter(bpod_trial, ones)

# get the times of the visual cue onsets
aux_vis = (event.Event & 'scan_id = "' + scansi + '"' & 'event_type LIKE "%vis%"').fetch("event_start_time")
# create a time series of ones, using the same length as the visual cue onsets
onesvis = np.ones_like(aux_vis) -1
axs[2].scatter(aux_vis, onesvis)

# get the times of the auditory cue onsets
aux_aud = (event.Event & 'scan_id = "' + scansi + '"' & 'event_type LIKE "%aud%"').fetch("event_start_time")
# create a time series of ones, using the same length as the auditory cue onsets
onesaud = np.ones_like(aux_aud) -2
axs[2].scatter(aux_aud, onesaud)

# get the times of the reward onsets

In [ ]:
def align_timeseries_and_find_non_matching_events(time_series_1, time_series_2):
    # Create event markers for each timeseries
    events_1 = np.zeros(int(np.ceil(max(time_series_1))))
    events_2 = np.zeros(int(np.ceil(max(time_series_2))))

    # Mark the events
    for event in time_series_1:
        events_1[int(event)] = 1
    for event in time_series_2:
        events_2[int(event)] = 1

    # Pad the shorter timeseries with zeros to make them the same length
    if len(events_1) > len(events_2):
        events_2 = np.pad(events_2, (0, len(events_1) - len(events_2)), 'constant')
    elif len(events_2) > len(events_1):
        events_1 = np.pad(events_1, (0, len(events_2) - len(events_1)), 'constant')

    # Perform cross-correlation
    correlation = correlate(events_1, events_2, mode='full')
    lag = np.argmax(correlation) - (len(events_2) - 1)

    # Shift the second series according to the lag
    aligned_series_2 = np.roll(events_2, -lag)[:len(events_1)]

    # Find the indices where events do not match
    non_matching_indices = np.where((events_1 == 1) & (aligned_series_2 == 0))[0]

    # Convert indices back to times
    non_matching_times = time_series_1[np.isin(time_series_1, non_matching_indices)]

    return non_matching_indices, non_matching_times, lag

# Align the two timeseries and find non-matching events
non_matching_indices, non_matching_times, alignment_lag = align_timeseries_and_find_non_matching_events(bpod_trial, aux_tone)

non_matching_indices, non_matching_times, alignment_lag

In [ ]:
aux_tone - aux_tone[0]

In [ ]:
bpod_trial-bpod_trial[1]

In [ ]:
non_matching_times, alignment_lag = align_timeseries_and_find_non_matching_events(bpod_trial, aux_tone)

In [ ]:
import numpy as np
from scipy.signal import correlate

def align_timeseries(series1, series2):
    """
    This function aligns two timeseries data and finds non-matching events.

    :param series1: First timeseries data as a numpy array.
    :param series2: Second timeseries data as a numpy array.
    :return: Indices of non-matching events in series1.
    """

    # Perform cross-correlation
    correlation = correlate(series1, series2, mode='full')
    # Find the lag that maximizes the cross-correlation
    lag = np.argmax(correlation) - (len(series2) - 1)

    # Align the series by shifting series2
    aligned_series2 = np.roll(series2, -lag)

    # Find non-matching events (where one series has an event and the other does not)
    non_matching_events = np.where((series1 != 0) != (aligned_series2[:len(series1)] != 0))[0]

    return non_matching_events, lag, aligned_series2

# Call the alignment function
non_matching_indices, calculated_lag, aligned_series2 = align_timeseries(bpod_trial, aux_tone)

print(f"Non-matching event indices in series1: {non_matching_indices}")
print(f"Lag for alignment: {calculated_lag}")

In [ ]:
aux_tone

In [ ]:
bpod_trial = bpod_object.session_data["TrialStartTimestamp"]
bpod_trial = np.concatenate(([1], bpod_trial))
ones = np.ones_like(bpod_trial)

ones2 = np.ones_like(aligned_series2)

aux_tone = (event.Event & 'scan_id = "' + scansi + '"' & 'event_type LIKE "%vis%"').fetch("event_start_time")
onesfive = np.ones_like(aux_tone) -.5

fig, axs = plt.subplots(2,figsize=(20, 6))
axs[0].scatter(aux_tone, onesfive+0.5)
axs[1].scatter(bpod_trial, ones)
axs[0].scatter(aligned_series2, ones2, c='r')

In [ ]:
from dtw import *
import matplotlib.pyplot as plt

# Create the alignment
alignment = dtw(aux_tone, bpod_trial, keep_internals=True)

# Plot the original time series
plt.figure(figsize=(20, 6))
plt.subplot(3, 1, 1)
plt.plot(aux_tone, label='aux_tone')
plt.plot(bpod_trial, label='bpod_trial')
plt.legend()

# Plot the warping path
plt.subplot(3, 1, 2)
alignment.plot(type="twoway",offset=-2)

# Plot the aligned time series
plt.subplot(3, 1, 3)
plt.plot(aux_tone[alignment.index1], label='aligned aux_tone')
plt.plot(bpod_trial[alignment.index2], label='aligned bpod_trial')
plt.legend()

plt.show()

In [ ]:
alignment.plot()

In [ ]:

def dtw(s, t, window):
    n, m = len(s), len(t)
    w = np.max([window, abs(n-m)])
    dtw_matrix = np.zeros((n+1, m+1))
    
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            dtw_matrix[i, j] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix


In [ ]:
time_series_1 = aux_tone
time_series_2 = bpod_trial
dtw(time_series_1, time_series_2, 100)

In [ ]:
import numpy as np

def dtw_alignment(ts_a, ts_b, d=lambda x, y: abs(x - y)):
    """Computes the Dynamic Time Warping (DTW) alignment between two time series.

    Args:
        ts_a: First time series (list or array).
        ts_b: Second time series (list or array).
        d: Distance function. Defaults to absolute difference.

    Returns:
        path: The optimal path for the alignment.
        DTW: The full DTW matrix.
    """
    # Create cost matrix with infinite values
    DTW = np.full((len(ts_a) + 1, len(ts_b) + 1), np.inf)
    DTW[0, 0] = 0

    # Populate the cost matrix
    for i in range(1, len(ts_a) + 1):
        for j in range(1, len(ts_b) + 1):
            cost = d(ts_a[i-1], ts_b[j-1])
            DTW[i, j] = cost + min(DTW[i-1, j],    # insertion
                                   DTW[i, j-1],    # deletion
                                   DTW[i-1, j-1])  # match

    # Backtrack to find the optimal path
    path = []
    i, j = len(ts_a), len(ts_b)
    while i > 0 and j > 0:
        path.append((i-1, j-1))
        min_index = np.argmin((DTW[i-1, j], DTW[i, j-1], DTW[i-1, j-1]))
        if min_index == 0:
            i -= 1
        elif min_index == 1:
            j -= 1
        else:  # min_index == 2
            i -= 1
            j -= 1

    # Include the start cell in the path
    path.append((0, 0))

    path.reverse()
    return path, DTW

# Example time series data
ts_a = aux_tone
ts_b = bpod_trial

# Get the alignment and DTW matrix
alignment_path, DTW_matrix = dtw_alignment(ts_a, ts_b)

# The alignment path is returned as tuples of aligned indices
alignment_path


In [ ]:
bpod_trial

In [ ]:
aux_tone

## Run image processing jobs


In [19]:
imaging.Processing.populate(**populate_settings)

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

using TorchFFT
{'data_path': ['/datajoint-data/data/tobiasr/RN_OPI-1681_2023-05-25_scan9FI8ALGO_sess9FI8ALGO'], 'tiff_list': ['/datajoint-data/data/tobiasr/RN_OPI-1681_2023-05-25_scan9FI8ALGO_sess9FI8ALGO/scan9FI8ALGO_RN_OPI-1681_02029.tif']}
tif
** Found 1 tifs - converting to binary **
time 8.83 sec. Wrote 4550 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
NOTE: applying classifier /home/tobiasr/adamacs/user_data/s2p_classifiers/TR_V1_mini2p_classifier.npy
----------- REGISTRATION
NOTE: estimated bidiphase offset from data: -3 pixels
Reference frame, 10.32 sec.


### Curation

In [ ]:
imaging.Curation().create1_from_processing_task({'session_id': sessi, 'scan_id': scansi, "paramset_idx": selected_s2pparms_index, "manual_curation": 0})

imaging.MotionCorrection.populate(**populate_settings)

imaging.Segmentation.populate(**populate_settings)

imaging.MaskClassification.populate(**populate_settings)

imaging.Fluorescence.populate(**populate_settings)

imaging.Activity.populate(**populate_settings)

## Pose estimation

In [ ]:
dj.Diagram(model) + dj.Diagram(equipment)

In [ ]:
aux_setup_typestr = (scan.ScanInfo() & 'scan_id = "' + scansi + '"').fetch("userfunction_info")[0]
selected_DLCmodel = 'Head_orientation-NK-2023-07-17'
print('- - - -')
print('DLC pose estimation:', scansi)

# insert TOP movie into model table
scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0] 
moviepath = str(list(pathlib.Path((scan.ScanPath() & scan_key).fetch("path")[0]).glob("*top*.mp4*"))[0])

key = {'session_id': scan_key["session_id"],
    'recording_id': scan_key["scan_id"], 
    'camera': "mini2p1_top", # Currently 'scanner' due to in equipment tables
    }
model.VideoRecording.insert1(key, skip_duplicates=True)

key.update({'file_path': moviepath,
            'file_id': 0})  #INCREMENT FILE_ID WITH CAM NUMBER?

model.VideoRecording.File.insert1(key, ignore_extra_fields=True, skip_duplicates=True)
model.RecordingInfo.populate()

In [ ]:

model.VideoRecording.File * model.VideoRecording() &  'recording_id = "' + scansi + '"'

In [ ]:
key =  (model.VideoRecording & f'recording_id="{scansi}"').fetch1('KEY')
key.update({'model_name': selected_DLCmodel, 'task_mode': 'trigger'}) 
key      


In [ ]:
# INSERT pose estimation task
model.PoseEstimationTask.insert_estimation_task(key, key["model_name"], analyze_videos_params={'save_as_csv':True, 'dynamic':(True,.5,60)}) # dynamic cropping

In [ ]:
model.PoseEstimationTask() * model.VideoRecording * model.VideoRecording.File()

In [ ]:
model.RecordingInfo()

In [ ]:
# run pose estimation
model.PoseEstimation.populate()

## Cleaning. Use with caution!

In [ ]:
# subject.Subject.delete()
# session.Session.delete()
# imaging.Processing.delete()
# imaging.Curation.delete()
# event.Event.delete()
# event.BehaviorRecording.delete()

In [19]:
# (session.Session & "session_id LIKE 'sess9FKJ0TLI%'").delete()
# (session.Session & "session_datetime >= '2023-10-01'").delete()
(imaging.ProcessingTask & "session_id LIKE 'sess9FI8ALGO%'").delete()
# (subject.Subject & "subject = 'ROS-1571'").delete()
# # subject.Subject.delete()
# # session.Session.delete()
# # imaging.Processing.delete()
# # imaging.Curation.delete()
# # event.Event.delete()
# # event.BehaviorRecording.delete()

[2023-11-09 18:15:11,796][INFO]: Deleting 2 rows from `tobiasr02_imaging`.`processing_task`
[2023-11-09 18:15:15,290][INFO]: Deletes committed.


2

In [ ]:
session.Session.delete()

In [ ]:
# session.Session.drop()
# scan.Scan.drop()
# imaging.Processing.drop()
# imaging.Curation.drop()